In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import rdDepictor
import numpy as np
import copy
from rdkit.Chem import Descriptors, rdMolDescriptors, Crippen
from rdkit.Chem import ResonanceMolSupplier, ResonanceFlags
import pandas as pd
from collections import defaultdict
from pandas.plotting import parallel_coordinates
import matplotlib.pyplot as plt
from rdkit.Chem import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import rdMolDraw2D
from cli.molslepa import DosEstimation
rdDepictor.SetPreferCoordGen(True)
IPythonConsole.drawOptions.minFontSize=20

vocabulary=[
    'C1=CC=CC=C1', 'NC=O', 'C1=CSC=N1', 'C1=COC=C1', 'C1=CN=CN=C1', 
    'C1CCCCCC1', 'C1CNC1', 'CC(C)C', 'C1=NC=NO1', 'C1=CNN=N1', 
    'C1CCOCC1', 'C1CNCCN1', 'C1=NN=CS1', 'C1=CC2=NC=CN2C=C1','FCF', 
    'CNC(N)=O', 'C1=CCNCC1', 'C1=CSN=C1', 'C=CC=O', 'CCOC', 
    'OC(F)F', 'C1CC1', 'CCN(C)C', 'COC(C)=O', 'CCS', 
    'C1=CN=C2C=CC=CC2=C1', 'C=CC(N)=O', 'CC#N', 'C1C2CC3CC1CC(C2)C3', 'N[SH](=O)=O'
    ]

In [16]:
def read_true_npy(thres_id, func):
    thresholds = [0.01, 0.05, 0.1, 0.5, 1, 2, 5]
    gt = np.load('results/bruteforce_gap_vocab30_K4_'+func+'_'+str(thresholds[thres_id])+'.npy')
    return gt

In [17]:
def read_allenergy(num_particle, max_beta, func, seed):  
    allenergy = np.load('results/molslepa_vocab30_K4/allenergy_gap_'+func+'_T'+str(max_beta)+'_s'+str(seed)+'_nump'+str(num_particle)+'_iter'+str(numiter)+'.npy')
    allseq = np.load('results/molslepa_vocab30_K4/allseq_gap_'+func+'_T'+str(max_beta)+'_s'+str(seed)+'_nump'+str(num_particle)+'_iter'+str(numiter)+'.npy')
    return allseq, allenergy

In [19]:
def epa_counter(gt, num_particle, max_beta, func, thresid, seed):   
    allseq, allenergy = read_allenergy(num_particle, max_beta, func,seed)
    betas = np.linspace(0, max_beta, len(allseq_all))
    counter = dos.multi_histogram(betas, allseq, allenergy, len(vocabulary), thresholds[thres_id])
    heldist = hellinger(counter, gt)
    return counter, heldist

In [20]:
def es_counter(gt, num_particle, func, thres_id, seed):
    allenergy = np.load('results/es_vocab30_K4/allenergy_gap_'+func+'_'+str(num_particle)+'_s'+str(seed)+'.npy')
    allseq = np.load('results/es_vocab30_K4/allseq_gap_'+func+'_'+str(num_particle)+'_s'+str(seed)+'.npy')
    vocab_size = len(vocabulary)
    vocab_len = 4
    thres = thresholds[thres_id]
    counter = np.zeros(len(allseq[0][0]))
    numiter = 20
    for biter in range(numiter):
        matches= [allseq[biter][i] for i,x in enumerate(allenergy[biter]) if x <= thres]
        matchenergy = [allenergy[biter][i] for i,x in enumerate(allenergy[biter]) if x <= thres] 
        for i in range(len(matches)):
            num_vocab = np.nonzero(matches[i])[0]
            for j in num_vocab:  
                for _ in range(int(matches[i][j])):  
                    counter[j] += 1
    counter /= np.sum(counter)
    heldist = hellinger(counter, gt)
    return counter, heldist

In [21]:
def rs_counter(gt, thres_id, seed):
    allenergy = np.load('results/random_vocab30_K4/allenergy_gap_'+str(num_particle*numiter)+'_s'+str(seed)+'.npy')
    allseq = np.load('results/random_vocab30_K4/allseq_gap_'+str(num_particle*numiter)+'_s'+str(seed)+'.npy')
    if func == 'max':
        allenergy = -allenergy
    thres = thresholds[thres_id]
    counter = np.zeros(len(allseq[0]))
    for i in range(len(allseq)):
        if allenergy[i] < thres:
            num_vocab = np.nonzero(allseq[i])[0]
            for n in num_vocab:
                for _ in range(int(allseq[i][n])):
                    counter[n] += 1
    counter = counter / np.sum(counter)
    heldist = hellinger(counter, gt) 
    return counter, heldist

In [23]:
def hellinger(p, q):
    return np.linalg.norm(np.sqrt(p)-np.sqrt(q))/np.sqrt(2)

In [24]:
def conjugated_group_number(ms):
    suppl = ResonanceMolSupplier(ms, ResonanceFlags.KEKULE_ALL) 
    return [ResonanceMolSupplier.GetNumConjGrps(suppl), 1-ResonanceMolSupplier.GetNumConjGrps(suppl)]

In [25]:
def aromatic_atom_number(mh):
    m = Chem.RemoveHs(mh)
    num_bonds = m.GetNumBonds()
    num_aromatic_bonds = 0
    for bond in m.GetBonds():
        if bond.GetIsAromatic():
            num_aromatic_bonds += 1
    ARR = num_aromatic_bonds/num_bonds
    return ARR

In [26]:
def aromatic_ring_number(ms):
    arom_ring_num = Descriptors.NumAromaticRings(ms)
    ring_num = rdMolDescriptors.CalcNumRings(ms)
    if arom_ring_num==ring_num and ring_num != 0:
        return [1,0]
    else:
        return [0,1]

In [27]:
def sp_number(ms):
    sp1_num = len(ms.GetSubstructMatches(Chem.MolFromSmarts('[^1]')))
    sp2_num = len(ms.GetSubstructMatches(Chem.MolFromSmarts('[^2]')))
    sp3_num = len(ms.GetSubstructMatches(Chem.MolFromSmarts('[^3]')))
    return sp2_num, sp3_num

In [134]:
def plotly_parallel_coordinates(counter_epa, counter_es, counter_rs, gt):
    data = defaultdict(list)
    index = np.argsort(-gt)
    gt = [gt[i] for i in index]
    vocabulary_sort = [vocabulary[i] for i in index]
    counter_epa = [counter_epa[i] for i in index]
    counter_es = [counter_es[i] for i in index]
    counter_rs = [counter_rs[i] for i in index]

    for i in range(len(gt)):
        data[vocabulary_sort[i]].append(gt[i])
    data['method'].append('Brute Force')

    for i in range(len(counter_epa)):
        data[vocabulary_sort[i]].append(counter_epa[i])
    data['method'].append('MolSLEPA')
    
    for i in range(len(counter_rs)):
        data[vocabulary_sort[i]].append(counter_rs[i])
    data['method'].append('Random')
    
    for i in range(len(counter_es)):
        data[vocabulary_sort[i]].append(counter_es[i])
    data['method'].append('ES')

    df = pd.DataFrame(data)

    fig,ax = plt.subplots(figsize=(12, 6))
    parallel_coordinates(df,'method', ax=ax, color=(gt_color, epa_color, rs_color, es_color), axvlines=False, linewidth=3)
    plt.scatter(x=range(len(vocabulary_sort)), y=gt, color=gt_color, alpha=0.8)
    plt.scatter(x=range(len(vocabulary_sort)), y=counter_epa, color=epa_color, alpha=0.8)
    plt.scatter(x=range(len(vocabulary_sort)), y=counter_rs, color=rs_color, alpha=0.8)
    plt.scatter(x=range(len(vocabulary_sort)), y=counter_es, color=es_color, alpha=0.8)
    plt.grid(linestyle = '--', alpha=0.5, linewidth = 0.5)
    plt.text(19, 0.365, s="Maximization, threshold=1%", size=17.5, family='Arial', weight='bold')
    plt.legend(loc='upper right', bbox_to_anchor=(0.97, 0.87), prop={'family':'Arial', 'size': 13})

    ax.set_ylabel('Saliency', fontsize=15,  fontfamily='Arial')
    ax.set_xticklabels(labels=vocabulary_sort, fontsize=12, rotation=75, fontfamily='Arial')
    ax.set_ylim(0, 0.4, 0.05)

    plt.tight_layout()
    plt.savefig('pc'+str(func)+'.pdf', bbox_inches='tight')
    plt.show()

In [141]:
thres_id = 4
max_beta = 3
seed = 3
numiter = 20
num_particle = 100
func = 'max'
gt_color = '#999999'
es_color = '#82B0D2'
epa_color = '#FA7F6F'
rs_color = '#F5B971'
betas = np.linspace(0, max_beta, numiter)
dos = DosEstimation()
thresholds = [-11.052256908920361, -9.818027661692127, -9.563187576028112, -8.740377669536942, -8.32855880836117, -7.720552925235464, -7.058249581258306]
# thresholds = [2.961448137576962, 3.1749314572896465, 3.2472055393168837, 3.4329222892415676, 3.533781962090044, 3.6524556344702566, 3.8395426085453024]

In [ ]:
gt = read_true_npy(thres_id, func)
es_count, es_dist = es_counter(gt, num_particle, func, thres_id, 3)
epa_count, epa_dist = epa_counter(gt, num_particle, max_beta, func, thres_id, seed)
rs_count, rs_dist = rs_counter(gt, thres_id, 3)

In [ ]:
plotly_parallel_coordinates(epa_count, es_count, rs_count, gt)

In [ ]:
import matplotlib as mpl
from matplotlib.font_manager import FontProperties
import math
seeds = 5
rs_dist_mean = []
es_dist_mean = []
epa_dist_mean = []
rs_dist_err = []
es_dist_err = []
epa_dist_err = []
for thres_id in range(7):
    rs_dist_list = []
    es_dist_list = []
    epa_dist_list = []
    gt = read_true_npy(thres_id, func)
    for s in range(seeds):
        rs_count, rs_dist = rs_counter(gt, thres_id, s)
        es_count, es_dist = es_counter(gt, num_particle, func, thres_id, s)
        epa_count, epa_dist = epa_counter(gt, num_particle, max_beta, func, thres_id, s)
        if np.isnan(rs_dist):
            rs_dist = 1
        rs_dist_list.append(rs_dist)
        es_dist_list.append(es_dist)
        epa_dist_list.append(epa_dist)
    rs_dist_mean.append(np.mean(rs_dist_list))
    es_dist_mean.append(np.mean(es_dist_list))
    epa_dist_mean.append(np.mean(epa_dist_list))
    rs_dist_err.append(np.std(np.array(rs_dist_list)))
    es_dist_err.append(np.std(np.array(es_dist_list)))
    epa_dist_err.append(np.std(np.array(epa_dist_list)))

In [ ]:
fig = plt.figure(figsize=(5,6))
x = np.arange(0,21,3)
y = [epa_dist_mean, rs_dist_mean, es_dist_mean]
tag = ['MolSLEPA', 'Random', 'ES']
color = [epa_color, rs_color, es_color]
std = [epa_dist_err, rs_dist_err, es_dist_err]
tick_label=['0.01', '0.05', '0.1','0.5','1','2', '5']
for i in range(3):
    upper = np.array(y[i]) + np.array(std[i])
    lower = np.array(y[i]) - np.array(std[i])
    plt.fill_between(x, upper, lower, alpha=0.2, color=color[i])
    plt.scatter(x,y[i], label=tag[i], color=color[i])
    plt.plot(x,y[i], color=color[i], linewidth=2)

plt.legend(loc='best', prop={'family':'Arial', 'size': 13})
plt.ylabel('Hellinger distance', fontsize=15,  fontfamily='Arial')
plt.xlabel('Threshold(%)', fontsize=15,  fontfamily='Arial')

plt.ylim([0.0,1])
plt.xticks(x,tick_label,fontsize=12)
plt.savefig('Hellinger distance.pdf', bbox_inches='tight')

In [114]:
func = 'min'
max_beta = 5
rs_best_property_all = []
es_best_property_all = []
epa_best_property_all = []
for seed in range(5):
    rs_best_property = []
    es_best_property = []
    epa_best_property = []
    rs_allenergy = np.load('results/random_vocab30_K4/allenergy_gap_'+str(num_particle*numiter)+'_'+str(seed)+'.npy')
    es_allenergy = np.load('results/es_vocab30_K4/allenergy_gap_'+func+'_'+str(num_particle)+'_'+str(seed)+'.npy').flatten()
    epa_allenergy = np.load('results/molslepa_vocab30_K4/allenergy_gap_'+func+'_T'+str(max_beta)+'_s'+str(seed)+'_nump'+str(num_particle)+'_iter'+str(numiter)+'.npy').flatten()
    for num in range(400, len(rs_allenergy)+1, 200):
        if func == 'max':
            rs_best_property.append(np.max(rs_allenergy[:num]))
            es_best_property.append(np.max(es_allenergy[:num]))
            epa_best_property.append(np.max(epa_allenergy[:num]))
        elif func == 'min':
            rs_best_property.append(np.min(rs_allenergy[:num]))
            es_best_property.append(np.min(es_allenergy[:num]))
            epa_best_property.append(np.min(epa_allenergy[:num]))
    rs_best_property_all.append(rs_best_property)
    es_best_property_all.append(es_best_property)
    epa_best_property_all.append(epa_best_property)
rs_best_property_all = np.array(rs_best_property_all)
es_best_property_all = np.array(es_best_property_all)
epa_best_property_all = np.array(epa_best_property_all)
    

In [ ]:
fig = plt.figure(figsize=(5,6))
x = np.arange(400,2001,200)
y = [epa_best_property_all.mean(axis=0), rs_best_property_all.mean(axis=0), es_best_property_all.mean(axis=0)]
tag = ['MolSLEPA', 'Random', 'ES']
color = [epa_color, rs_color, es_color]
std = [epa_best_property_all.std(axis=0), rs_best_property_all.std(axis=0), es_best_property_all.std(axis=0)]
for i in range(3):
    upper = np.array(y[i]) + np.array(std[i])
    lower = np.array(y[i]) - np.array(std[i])
    plt.fill_between(x, upper, lower, alpha=0.2, color=color[i])
    plt.scatter(x,y[i], label=tag[i], color=color[i])
    plt.plot(x,y[i], color=color[i], linewidth=2)

plt.ylabel('Best property (eV)', fontsize=15,  fontfamily='Arial')
plt.xlabel('Number of samples', fontsize=15,  fontfamily='Arial')
plt.ylim([2.5, 3.6])
# plt.ylim([8.8, 13.1])
plt.legend(loc='best', prop={'family':'Arial', 'size': 13})
plt.savefig('Best_property.pdf', bbox_inches='tight')